Adjust workflow
*change load classes to the utils
* add in the youtube download to utils XX
* add changes for above two changes the main.py XX
* change main.py to train.py so you can full automate to CLI easiest
* add in the classes to the labels visualizations XX
* add uses to it 
* add to hugging face/Snap AR
* attempt to use cv2 to see when basketballs intersect with rims XX

images outside of youtube nba footage come from: [text](https://stock.adobe.com/search?filters%5Bcontent_type%3Aphoto%5D=1&filters%5Bcontent_type%3Aillustration%5D=1&filters%5Bcontent_type%3Azip_vector%5D=1&filters%5Bcontent_type%3Avideo%5D=1&filters%5Bcontent_type%3Atemplate%5D=1&filters%5Bcontent_type%3A3d%5D=1&filters%5Bcontent_type%3Aaudio%5D=0&filters%5Binclude_stock_enterprise%5D=0&filters%5Bis_editorial%5D=0&filters%5Bfree_collection%5D=0&filters%5Bcontent_type%3Aimage%5D=1&k=basketball+hoop&order=relevance&safe_search=1&limit=100&search_page=21&search_type=pagination&get_facets=0)

In [18]:
import torch

print(torch.cuda.is_available())

True


In [7]:
!git clone https://github.com/nwojke/deep_sort.git

fatal: destination path 'deep_sort' already exists and is not an empty directory.


In [8]:
import os

# Get and print the current working directory before changing it
current_directory = os.getcwd()
print("Current working directory before change:", current_directory)

# Define the new directory
new_directory = r"C:\Users\ghadf\vscode_projects\venv_projects\Pytorch\bball_instanceseg"
parent_directory = os.path.dirname(new_directory)

# Change the current working directory to the new directory
os.chdir(new_directory)

# Get and print the current working directory after changing it
current_directory = os.getcwd()
print("Current working directory after change:", current_directory)


Current working directory before change: C:\Users\ghadf\vscode_projects\venv_projects\Pytorch\bball_instanceseg
Current working directory after change: C:\Users\ghadf\vscode_projects\venv_projects\Pytorch\bball_instanceseg


In [9]:
import torch
from torch import nn

# Note: this notebook requires torch >= 1.10.0
print(torch.__version__)
print("CUDA available: ", torch.cuda.is_available())


# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

2.2.0+cu118
CUDA available:  True


'cuda'

In [10]:
import os

def create_directory(dir_path):
    """Create a directory if it does not exist."""
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

#create going_modular repository
create_directory("going_modular")

In [11]:
# Download required files from torchvision
import requests

def download_files(urls):
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            with open(url.split("/")[-1], 'wb') as file:
                file.write(response.content)
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")

urls = [
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py"
]
download_files(urls)

In [12]:
%%writefile going_modular/utils.py
#!pip install roboflow


#from roboflow import Roboflow
#rf = Roboflow(api_key="htpcxp3XQh7SsgMfjJns")
#project = rf.workspace("ai-79z1a").project("basketball_child")
#dataset = project.version(6).download("coco-segmentation")


from roboflow import Roboflow
import torch
import requests
import yt_dlp
import os

def download_videos_from_youtube(video_urls, output_path):
    """
    Downloads videos from YouTube.

    Args:
    video_urls (list): List of YouTube video URLs.
    output_path (str): Directory where videos will be saved.

    Returns:
    tuple: A tuple containing lists of successful and failed downloads.
    """

    ydl_opts = {
        'format': 'best',
        'outtmpl': output_path + '/%(title)s.%(ext)s',
        'quiet': True
    }

    failed_downloads = []
    successful_downloads = []

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        for url in video_urls:
            try:
                ydl.download([url])
                print(f"Successfully downloaded {url}")
                successful_downloads.append(url)
            except Exception as e:
                print(f"Failed to download {url}: {e}")
                failed_downloads.append(url)

    return successful_downloads, failed_downloads


def get_device():
    return "cuda" if torch.cuda.is_available() else "cpu"


def get_project(api_key, workspace, project_name, version):
    rf = Roboflow(api_key=api_key)
    project = rf.workspace(workspace).project(project_name)
    dataset = project.version(version).download("coco-segmentation")
    return dataset

def download_files(urls):
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            with open(url.split("/")[-1], 'wb') as file:
                file.write(response.content)
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")


def construct_dataset_paths(project_name, version):
    base_path = f"{project_name}-{version}"
    train_annotation_path = f"{base_path}/train/_annotations.coco.json"
    valid_annotation_path = f"{base_path}/valid/_annotations.coco.json"
    test_annotation_path = f"{base_path}/test/_annotations.coco.json"

    train_root_dir = f"{base_path}/train"
    valid_root_dir = f"{base_path}/valid"
    test_root_dir = f"{base_path}/test"

    return train_annotation_path, valid_annotation_path, test_annotation_path, train_root_dir, valid_root_dir, test_root_dir

def create_directory(dir_path):
    """Create a directory if it does not exist."""
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


Overwriting going_modular/utils.py


In [13]:
%%writefile going_modular/coco_dataset.py

import json
import os
import numpy as np
from PIL import Image, ImageDraw
import torch
from torchvision.transforms.v2 import functional as F
from torchvision import tv_tensors

class CustomCocoDataset(torch.utils.data.Dataset):
    def __init__(self, annotation_path, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms

        with open(annotation_path) as f:
            self.annotations = json.load(f)

        # Filter out images without annotations
        annotated_images = []
        for img in self.annotations['images']:
            image_id = img['id']
            anns = [ann for ann in self.annotations['annotations'] if ann['image_id'] == image_id]
            if len(anns) > 0:
                annotated_images.append(img)

        self.image_ids = [img['id'] for img in annotated_images]

        # Update the self.annotations['images'] to include only annotated images
        self.annotations['images'] = annotated_images
        
        #print("Number of images:", len(self.annotations['images']))
        #print("Sample image entry:", self.annotations['images'][0])

    def __len__(self):
        return len(self.annotations['images'])

    def __getitem__(self, idx):
        img_info = self.annotations['images'][idx]
        image_id = img_info['id']
        
        img_path = os.path.join(self.root_dir, img_info['file_name'])
        img = Image.open(img_path).convert("RGB")
        img_tensor = F.to_tensor(img)
        #print("Image size (PIL):", img.size)
        #print("Image shape (tensor):", img_tensor.shape)

        anns = [ann for ann in self.annotations['annotations'] if ann['image_id'] == image_id]
        #print("Number of annotations for this image:", len(anns))

        boxes = [ann['bbox'] for ann in anns]  # bbox format: [x_min, y_min, width, height]
        # Convert from XYWH to XYXY format
        boxes = [[box[0], box[1], box[0] + box[2], box[1] + box[3]] for box in boxes]
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = [ann['category_id'] for ann in anns]
        labels = torch.as_tensor(labels, dtype=torch.int64)
        #print("Boxes shape:", boxes.shape)
        #print("Labels:", labels)
        # Debug print
        #print(f"Boxes shape for image {idx}: {boxes.shape}")

        masks = []
        for ann in anns:
            if 'segmentation' in ann and isinstance(ann['segmentation'], list):
                for seg in ann['segmentation']:
                    mask_img = Image.new('L', (img_info['width'], img_info['height']), 0)
                    ImageDraw.Draw(mask_img).polygon(seg, outline=1, fill=1)
                    mask = np.array(mask_img)
                    masks.append(mask)
        masks = torch.as_tensor(np.array(masks), dtype=torch.uint8) if masks else torch.zeros((0, img_info['height'], img_info['width']), dtype=torch.uint8)
        #print("Masks shape:", masks.shape)

        areas = [ann['area'] for ann in anns]
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = [ann['iscrowd'] for ann in anns]
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        # Convert masks to Mask format
        masks = tv_tensors.Mask(masks)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id  # Changed to integer
        target["area"] = areas
        target["iscrowd"] = iscrowd

        #print("Target:", target)

        if self.transforms is not None:
            img_tensor, target = self.transforms(img_tensor, target)

        return img_tensor, target





Overwriting going_modular/coco_dataset.py


In [14]:
%%writefile going_modular/visualization_utils.py
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F  # Add this import

# New function to visualize transformations
def visualize_transformation(dataset, idx):
    img, target = dataset[idx]
    transformed_img, transformed_target = dataset.transforms(img, target)
    original_img = F.to_pil_image(img)
    transformed_img = F.to_pil_image(transformed_img)

    plt.figure(figsize=(24, 6))
    # Original Image
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    for box in target["boxes"]:
        x_min, y_min, x_max, y_max = box.tolist()
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)
        #print(x_min, y_min, x_max, y_max)
    plt.title(f"Original Image - ID: {idx}")

    # Transformed Image
    plt.subplot(1, 2, 2)
    plt.imshow(transformed_img)
    for box in transformed_target["boxes"]:
        x_min, y_min, x_max, y_max = box.tolist()
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='b', facecolor='none')
        plt.gca().add_patch(rect)
        #print(x_min, y_min, x_max, y_max)
    plt.title(f"Transformed Image - ID: {idx}")
    plt.show()



def visualize_bbox(dataset, idx):
    img, target = dataset[idx]
    original_img = F.to_pil_image(img)

    plt.figure(figsize=(12, 6))
    plt.imshow(original_img)

    for box in target["boxes"]:  # Access the boxes directly
        x_min, y_min, x_max, y_max = box.tolist()
        # Debug print
        print(f"Visualizing BBox - xmin: {x_min}, ymin: {y_min}, xmax: {x_max}, ymax: {y_max}")
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)

    plt.title(f"Image with Bounding Boxes - ID: {idx}")
    plt.show()


Overwriting going_modular/visualization_utils.py


In [15]:
%%writefile going_modular/model_utils.py
import json
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def load_classes_from_json(file_path):
    """
    Loads the class names and their corresponding IDs from a COCO format JSON file.

    Args:
    file_path (str): Path to the JSON file.

    Returns:
    dict: A dictionary where keys are class IDs and values are class names.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Extracting classes and their IDs
    classes = {category['id']: category['name'] for category in data['categories']}
    return classes

# Usage example:
#classes = load_classes_from_json('basketball_child-6/test/_annotations.coco.json')
#print(classes)

# model_utils.py
def get_model_instance_segmentation(num_classes, hidden_layer=256):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    return model



Overwriting going_modular/model_utils.py


In [16]:
%%writefile going_modular/transforms.py
import torch  # Add this import statement
from torchvision.transforms import v2 as T
from torchvision.transforms import Compose, RandomHorizontalFlip, ToTensor, ConvertImageDtype

def get_transform(train):
    transforms = []
    #if train:
    #    transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)



Overwriting going_modular/transforms.py


In [17]:
%%writefile going_modular/engine.py
# train.py
import torch
import torchvision
from engine import train_one_epoch, evaluate
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator

def train_model(model, data_loader, data_loader_valid, device, num_epochs,
                lr=0.005, momentum=0.9, weight_decay=0.0005, step_size=3, gamma=0.1):
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        lr_scheduler.step()
        evaluate(model, data_loader_valid, device=device)

    #torch.save(model.state_dict(), 'results/models/model_weights.pth')


Overwriting going_modular/engine.py


In [6]:
%%writefile going_modular/process_video_check.py

import cv2
import torch
import numpy as np
from torchvision.transforms import v2 as T
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks


def intersects(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1.tolist()
    x2_min, y2_min, x2_max, y2_max = box2.tolist()
    return (x1_min < x2_max and x1_max > x2_min and y1_min < y2_max and y1_max > y2_min)

def process_video_check(video_path, model, device, classes, classes_to_track, threshold=0.5):
    model.eval()
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("Error opening video file")
        return

    score = 0  # Initialize the score counter

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_tensor = T.ToTensor()(frame).unsqueeze_(0).to(device)

        with torch.no_grad():
            prediction = model(frame_tensor)[0]

        pred_scores = prediction['scores']
        pred_boxes = prediction['boxes']
        pred_labels = prediction['labels']
        pred_masks = prediction['masks']

        keep = pred_scores > threshold
        pred_boxes = pred_boxes[keep]
        pred_labels = pred_labels[keep]
        pred_masks = pred_masks[keep]

        print(f"Original Frame Size: {frame.shape}")
        print(f"Pred Boxes before drawing: {pred_boxes}")

        if not keep.any():
            continue  # Skip this frame if no detections are kept

        # Convert numeric labels to class names
        pred_class_names = [classes[label.item()] for label in pred_labels]

        # Iterate through each pair of classes to track
        for class_pair in classes_to_track:
            class1_boxes = pred_boxes[[name == class_pair[0] for name in pred_class_names]]
            class2_boxes = pred_boxes[[name == class_pair[1] for name in pred_class_names]]

            # Check for intersections and update score
            for box1 in class1_boxes:
                for box2 in class2_boxes:
                    if intersects(box1, box2):
                        score += 1
                        print(f"Intersection detected between {class_pair[0]} and {class_pair[1]}, Score:", score)

        # Frame Tensor Conversion for Drawing
        frame_tensor = (255.0 * (frame_tensor - frame_tensor.min()) / (frame_tensor.max() - frame_tensor.min())).to(torch.uint8)
        frame_tensor = frame_tensor.squeeze().to(torch.uint8)

        # Draw bounding boxes and segmentation masks
        output_image = draw_bounding_boxes(frame_tensor, pred_boxes, labels=pred_class_names, colors="red")
        output_image = draw_segmentation_masks(output_image, (pred_masks > 0.7).squeeze(1), alpha=0.5, colors="blue")

        # Convert output image for displaying
        output_image = output_image.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
        output_image = np.clip(output_image, 0, 255)  # Ensure values are within 0-255

        # Draw score text
        cv2.putText(output_image, f'Score: {score}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        cv2.imshow('Frame', output_image)
        
        # Just before cv2.imshow
        print(f"Output Image Size: {output_image.shape}")

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
# process_video_check(video_path, model, device, classes, [('ball', 'rim')], threshold=0.5)


Overwriting going_modular/process_video_check_track.py


In [13]:
%%writefile going_modular/process_video_check_track.py
import cv2
import numpy as np
import torch
from torchvision.transforms.functional import to_tensor, convert_image_dtype
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import torchvision.transforms.functional as F
from bytetracker.kalman_filter import KalmanFilter
from torchvision.ops import box_convert

# Utility Functions
def tlwh_to_xyah(tlwh):
    x, y, w, h = tlwh
    center_x = x + w / 2
    center_y = y + h / 2
    aspect_ratio = w / h
    return np.array([center_x, center_y, aspect_ratio, h])

def xyah_to_tlwh(xyah):
    """
    Convert the state vector (including velocity or other components)
    back to the top-left x, y, width, height (tlwh) bounding box format.
    """
    # Ensure xyah only contains [center_x, center_y, aspect_ratio, height]
    if len(xyah) > 4:
        xyah = xyah[:4]  # Consider only the first 4 components for conversion
    center_x, center_y, aspect_ratio, h = xyah
    w = aspect_ratio * h
    tlwh = np.array([center_x - w / 2, center_y - h / 2, w, h])
    return tlwh

# Track Class
class Track:
    _id_count = 1
    
    def __init__(self, tlwh, score):
        self.kalman_filter = KalmanFilter()
        self.xyah = tlwh_to_xyah(tlwh)
        self.state, self.covariance = self.kalman_filter.initiate(self.xyah)
        self.track_id = Track._id_count
        Track._id_count += 1
        self.score = score
        self.hits = 1
        self.age = 1
        self.time_since_update = 0

    def predict(self):
        self.state, self.covariance = self.kalman_filter.predict(self.state, self.covariance)
        self.age += 1
        self.time_since_update += 1

    def update(self, tlwh):
        self.xyah = tlwh_to_xyah(tlwh)
        self.state, self.covariance = self.kalman_filter.update(self.state, self.covariance, self.xyah)
        self.hits += 1
        self.time_since_update = 0

    def get_tlwh(self):
        return xyah_to_tlwh(self.state[:4])

def iou(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    xi1, yi1 = max(x1, x2), max(y1, y2)
    xi2, yi2 = min(x1+w1, x2+w2), min(y1+h1, y2+h2)
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)
    box1_area, box2_area = w1*h1, w2*h2
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def adjust_boxes(boxes, scale_factor=(1.0, 1.0)):
    """Adjust bounding boxes by a scale factor."""
    adjusted_boxes = []
    for box in boxes:
        # Assuming box is in tlwh format
        x, y, w, h = box
        adjusted_box = [x * scale_factor[0], y * scale_factor[1], (x + w) * scale_factor[0], (y + h) * scale_factor[1]]
        adjusted_boxes.append(adjusted_box)
    return adjusted_boxes

def process_video_check_track(video_path, model, device, threshold=0.5):
    model.eval()
    cap = cv2.VideoCapture(str(video_path))
    tracks = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        original_width, original_height = frame.shape[1], frame.shape[0]
        frame_tensor = to_tensor(frame).unsqueeze_(0).to(device)
        with torch.no_grad():
            prediction = model(frame_tensor)[0]

        keep = prediction['scores'] > threshold
        boxes = prediction['boxes'][keep].cpu()
        scores = prediction['scores'][keep].cpu()

        # Ensure the loop for processing detections and adjusting boxes is correctly scoped
        for i, box in enumerate(boxes):
            det = box.numpy()  # Convert to numpy array
            score = scores[i].item()

            # Scale factor for adjusting boxes to the original frame size
            scale_factor = (original_width / frame_tensor.shape[3], original_height / frame_tensor.shape[2])
            adjusted_box = adjust_boxes([det], scale_factor)[0]
            adjusted_box_tensor = torch.tensor([adjusted_box], dtype=torch.float32)
            adjusted_tlbr = box_convert(adjusted_box_tensor, in_fmt='xywh', out_fmt='xyxy')

            matched = False
            for track in tracks:
                if iou(adjusted_tlbr.squeeze(0).numpy(), torch.tensor(track.get_tlwh()).unsqueeze(0).numpy()) > 0.5:
                    track.update(det)
                    matched = True
                    break

            if not matched:
                tracks.append(Track(det, score))

        # Prepare the frame for drawing
        frame_draw = convert_image_dtype(frame_tensor.squeeze(0), dtype=torch.uint8)

        # Correctly use adjusted boxes within this loop
        for track in tracks:
            tlwh = track.get_tlwh()
            track_box_adjusted = adjust_boxes([tlwh], scale_factor)[0]
            track_box_tensor = torch.tensor([track_box_adjusted], dtype=torch.float32)
            track_tlbr = box_convert(track_box_tensor, in_fmt='xywh', out_fmt='xyxy')
            frame_draw = draw_bounding_boxes(frame_draw, track_tlbr, colors="blue", width=3)

        # Convert tensor to numpy array for display
        output_image = frame_draw.permute(1, 2, 0).cpu().numpy()
        cv2.imshow('Frame with Tracks', output_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()




Overwriting going_modular/process_video_check_track.py


In [14]:
%%writefile train.py
import argparse
import json
from pathlib import Path
import torch
from torch.utils.data import random_split, DataLoader
from going_modular.utils import get_device, create_directory, get_project, download_videos_from_youtube
from going_modular.coco_dataset import CustomCocoDataset
from going_modular.model_utils import get_model_instance_segmentation
from going_modular.engine import train_model
from going_modular.transforms import get_transform
from going_modular.process_video_check_track import process_video_check_track
import utils

def load_classes_from_json(annotation_path):
    with open(annotation_path) as f:
        data = json.load(f)
    categories = data['categories']
    classes = {category['id']: category['name'] for category in categories}
    return classes

def split_dataset(dataset, split_ratio=0.8):
    total_size = len(dataset)
    train_size = int(total_size * split_ratio)
    valid_size = total_size - train_size
    train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(42))
    return train_dataset, valid_dataset

def main(args):
    # Create model path directory
    model_path = Path(args.model_path)
    create_directory(model_path)

    # Check if dataset is already downloaded
    project_folder = Path.cwd() / f'{args.project_folder_name}-{args.version}'
    print(f"Checking dataset at: {project_folder.absolute()}")

    if not project_folder.exists():
        print("Downloading dataset...")
        dataset = get_project(args.api_key, args.workspace, args.project_name, args.version)
    else:
        print("Project data already downloaded. Skipping download...")

    # Load classes from annotation file
    classes_path = project_folder / 'train' / '_annotations.coco.json'
    if not classes_path.exists():
        print(f"Error: Annotation file for the train dataset not found at {classes_path.absolute()}.")
        return
    classes = load_classes_from_json(classes_path)
    print("Classes loaded:", classes)

    # Initialize model
    num_classes = len(classes) + 1
    device = get_device()
    model = get_model_instance_segmentation(num_classes, hidden_layer=args.hidden_layer)
    model.to(device)

    # Load datasets
    datasets = {}
    data_loaders = {}
    for dtype in ['train', 'valid', 'test']:
        ann_path = project_folder / dtype / '_annotations.coco.json'
        img_dir = project_folder / dtype
        if ann_path.exists() and img_dir.exists():
            datasets[dtype] = CustomCocoDataset(str(ann_path), str(img_dir), transforms=get_transform(train=dtype=='train'))
            batch_size = 2 if dtype == 'train' else 1
            data_loaders[dtype] = DataLoader(datasets[dtype], batch_size=batch_size, shuffle=dtype=='train', num_workers=0, collate_fn=utils.collate_fn)
            print(f"{dtype.capitalize()} dataset loaded.")
        else:
            print(f"{dtype.capitalize()} dataset not found or incomplete. Skipping.")

    # Split dataset if valid is not loaded
    if args.mode == 'train' and 'train' in datasets and 'valid' not in datasets:
        train_dataset, valid_dataset = split_dataset(datasets['train'])
        data_loaders['train'] = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0, collate_fn=utils.collate_fn)
        data_loaders['valid'] = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)

        # Execute training
        if 'train' in data_loaders:
            print("Starting training process...")
            train_model(model, data_loaders['train'], data_loaders.get('valid'), device, args.num_epochs, lr=args.lr)
            model_file_path = model_path / 'model_weights.pth'
            torch.save(model.state_dict(), model_file_path)
            print(f"Model saved at: {model_file_path}")
        else:
            print("Training failed. No data loaders available.")

    # Process video if mode is 'process_video'
    if args.mode == 'process_video':
        video_filename = args.video_name if args.video_name.endswith('.mp4') else f"{args.video_name}.mp4"
        video_path = Path(args.data_path) / video_filename
        if not video_path.exists():
            print("Downloading video...")
            download_videos_from_youtube([args.video_url], str(Path(args.data_path)))
        
        if video_path.exists():
            model_file_path = model_path / 'model_weights.pth'
            if model_file_path.exists():
                model.load_state_dict(torch.load(str(model_file_path), map_location=device))
                #process_video_check_track(video_path, model, device, classes, [('Basketball', 'Hoop')], threshold=args.threshold)
                process_video_check_track(video_path, model, device, threshold=0.5)
            else:
                print("Model weights file not found. Please train the model first.")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train a model for object detection or process video")
    parser.add_argument('--api_key', type=str, default="your_roboflow_api_key", help='API key for Roboflow')
    parser.add_argument('--workspace', type=str, default="your_roboflow_workspace", help='Workspace name in Roboflow')
    parser.add_argument('--project_name', type=str, default="your_roboflow_project", help='Project name in Roboflow')
    parser.add_argument('--project_folder_name', type=str, default="your_roboflow_project_folder_name", help='Project folder name in Roboflow')
    parser.add_argument('--version', type=int, default=1, help='Version of the dataset in Roboflow')
    parser.add_argument('--hidden_layer', type=int, default=256, help='Hidden layer size for the MaskRCNN predictor')
    parser.add_argument('--lr', type=float, default=0.005, help='Learning rate')
    parser.add_argument('--num_epochs', type=int, default=10, help='Number of epochs to train the model')
    parser.add_argument('--video_url', type=str, default="https://www.youtube.com/watch?v=example_video_id", help='URL of the video to process')
    parser.add_argument('--video_name', type=str, default="your_youtube_video_name", help='Name of the video file (with extension) to process')
    parser.add_argument('--threshold', type=float, default=0.6, help='Detection threshold for process_video')
    parser.add_argument('--display_video', type=bool, default=True, help='Whether to display the video during processing')
    parser.add_argument('--data_path', type=str, default='results/data', help='Path to save downloaded data')
    parser.add_argument('--model_path', type=str, default='results/models', help='Path to save model weights')
    parser.add_argument('--mode', type=str, choices=['train', 'process_video'], default='train', help='Mode of operation: train or process_video')
    
    args = parser.parse_args()
    main(args)


Overwriting train.py


In [15]:
#example usage
!python train.py --api_key htpcxp3XQh7SsgMfjJns \
                --workspace basketball-formations \
                --project_name basketball-and-hoop-7xk0h \
                --project_folder_name basketball-and-hoop \
                --version 11 \
                --hidden_layer 256 \
                --lr 0.005 \
                --num_epochs 10 \
                --threshold 0.6 \
                --video_url "https://www.youtube.com/watch?v=kh7s2tGvswc&t=1s" \
                --video_name "Devin Booker Sets Record, Wins Three-Point Contest" \
                --mode process_video

#ex command line usage
#python train.py --api_key htpcxp3XQh7SsgMfjJns --workspace basketball-formations --project_name basketball-and-hoop-7xk0h --project_folder_name basketball-and-hoop --version 10 --hidden_layer 256 --lr 0.005 --num_epochs 1 --threshold 0.6 --video_url "https://www.youtube.com/watch?v=y8i6fsAXDZE" --video_name "The Best NBA 3 Point Contest Performances" --mode process_video


Checking dataset at: c:\Users\ghadf\vscode_projects\venv_projects\Pytorch\bball_instanceseg\basketball-and-hoop-11
Project data already downloaded. Skipping download...
Classes loaded: {0: 'Basketball-and-Hoop', 1: 'Backboard', 2: 'Basketball', 3: 'Hoop', 4: 'Player'}
Train dataset loaded.
Valid dataset loaded.
Test dataset loaded.


Traceback (most recent call last):
  File "c:\Users\ghadf\vscode_projects\venv_projects\Pytorch\bball_instanceseg\train.py", line 123, in <module>
    main(args)
  File "c:\Users\ghadf\vscode_projects\venv_projects\Pytorch\bball_instanceseg\train.py", line 100, in main
    process_video_check_track(video_path, model, device, threshold=0.5)
  File "c:\Users\ghadf\vscode_projects\venv_projects\Pytorch\bball_instanceseg\going_modular\process_video_check_track.py", line 112, in process_video_check_track
    if iou(adjusted_tlbr.squeeze(0).numpy(), torch.tensor(track.get_tlwh()).unsqueeze(0).numpy()) > 0.5:
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ghadf\vscode_projects\venv_projects\Pytorch\bball_instanceseg\going_modular\process_video_check_track.py", line 62, in iou
    x2, y2, w2, h2 = bbox2
    ^^^^^^^^^^^^^^
ValueError: not enough values to unpack (expected 4, got 1)
